# What is the True Normal Human Body Temperature? 

#### Background

The mean normal body temperature was held to be 37$^{\circ}$C or 98.6$^{\circ}$F for more than 120 years since it was first conceptualized and reported by Carl Wunderlich in a famous 1868 book. But, is this value statistically correct?

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np

df = pd.read_csv('data/human_body_temperature.csv')

import seaborn as sns
sns.set()

## Is the distribution of body temperatures normal?

In [ ]:
sns.distplot(df['temperature'])

In [ ]:
sns.distplot(df['temperature'],
             hist_kws=dict(cumulative=True),
             kde_kws=dict(cumulative=True))

In [ ]:
from scipy import stats

k2, p = stats.normaltest(df['temperature'])
print("p = {:g}".format(p))

print("The data is normally distributed!")

## Is the sample size large? Are the observations independent?

##### Yes, sample size > 20 therefore it is large enough. Also the observations were from different individuals and thus independant

## Is the true population mean really 98.6 degrees F?

### The hypothesis is it occured by random chance  and the null hypothesis is the true mean is 98.6.

In [ ]:
mu = 98.6
mean = df['temperature'].mean()
temp_shifted = df['temperature'] - mean + mu

### The sample mean is 5.45 standard error units below the assumed population mean of 98.6 F

In [ ]:
def t_stat(data, mu=98.6):
    """Calculate t-statistic"""
    return (np.mean(data) - mu) / (np.std(data) / np.sqrt(len(data)))

t_obs = t_stat(df['temperature'], mu)

def bootstrap_replicate_1d(data, func):
    """Draw one bootstrap replicate for a 1-d array of data"""
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw (size) number of bootstrap replicates."""
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)
    return bs_replicates


bs_t_reps = draw_bs_reps(temp_shifted, t_stat, size=10000)


# Compute the p-value
p_t_value = np.sum(np.abs(bs_t_reps) >= np.abs(t_obs)) / len(bs_t_reps)


print('t-statistic for the observations:' + str(t_obs))
print('p-value: {:.2e}'.format(p_t_value))

### We must reject the null hypothesis the true population mean is not 98.6

### Since there is only one set of data a one sample test is approitate. 

In [ ]:
# Define z-statistic function
def z_stat(data, mu=98.6, sigma=0.7303577789050377):
    """Calculate z-statistic"""
    return (np.mean(data) - mu) / (sigma / np.sqrt(len(data)))

z_obs = z_stat(df['temperature'], mu) 

bs_z_reps = draw_bs_reps(temp_shifted, z_stat, size=10000)

p_z_value = np.sum(np.abs(bs_z_reps) >= np.abs(z_obs)) / len(bs_z_reps)


print('z-statistic for the observations: ' + str(z_obs))
print('p-value: {:.2e}'.format(p_z_value))

### Both the z and t test produce the same results.

## Does this hold for a small sample size of 10?

In [ ]:
sample = np.random.choice(df['temperature'], size=10)

mean = sample.mean()
temp_shifted = sample - mean + mu

z_obs = z_stat(sample) 

bs_z_reps = draw_bs_reps(temp_shifted, z_stat, size=10000)

p_z_value = np.sum(np.abs(bs_z_reps) >= np.abs(z_obs)) / len(bs_z_reps)


print('z-statistic for the observations: ' + str(z_obs))
print('p-value: {:.2e}'.format(p_z_value))

### We still reject the null hypothesis.

## When is someone's temperature to be "abnormal"

In [ ]:
# Margin of Error
z_95 = 1.96 # z-value corresponding to 95% confidence interval
mean = df['temperature'].mean()
margin_e = z_95 * (df['temperature'].std()/len(df['temperature']))
print('The margin of error on the mean temperature is: ' + str(margin_e) + 'deg. F')


# 95% Confidence Interval on the mean 
conf_int = np.percentile(df['temperature'], [2.5, 97.5])
print('95% of the data fall between: ' + str(conf_int[0]) + ' - ' + str(conf_int[1]) + 'deg. F')


# 95% Prediction Interval
pred_int = z_95 * df['temperature'].std()/np.sqrt(1)  
print('95% prediction interval ' + str(mean - pred_int) + ' - ' + str(mean + pred_int )+ 'deg. F')

### Any observation outside of the predicition interval is consider adnormal.

## Do male and females have a significant difference in normal temperature?

In [ ]:
# Observations by gender.
females = df[df['gender'] == 'F']
males = df[df['gender'] == 'M']

# Descriptive statistics 
print('Females')
print(females['temperature'].describe())
print(' ')
print('Males')
print(males['temperature'].describe())
f_mean = np.mean(females['temperature'])
m_mean = np.mean(males['temperature'])

## Null hypothesis: The distributions of temperatures are identical.

In [ ]:
# Define test statistic 
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays"""
    return np.mean(data_1) - np.mean(data_2)


# Compute test statistic for observed data
diff_obs = diff_of_means(females['temperature'], males['temperature'])


# Draw permutation replicates
perm_reps = np.empty(10000)
for i in range(len(perm_replicates)): 
    # Permute the data
    both = np.concatenate((females['temperature'], males['temperature']))
    both_perm = np.random.permutation(both)
    perm_females = both_perm[:len(females)]
    perm_males = both_perm[len(females):]
    
    #Draw replicates
    perm_reps[i] = diff_of_means(perm_females, perm_males)


# Compute the p-value
p_value = np.sum(abs(perm_replicates) >= abs(diff_obs)) / len(perm_replicates)

print('Difference: ' + str(diff_obs) + 'deg. F' )
print('p-value: ' + str(p_value))